# describe_duration.ipynb

This notebook provides descriptive statistics (median and IQR) of the duration of sick notes associated with long COVID.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns

from functools import reduce
from glob import glob

pd.options.mode.chained_assignment = None

## Import and Clean Data

In [2]:
# Function to parse string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [3]:
# Read in and append input files
li = []

for file in glob('../output/cohorts/cohort_rates*.dta'):
    df_temp = pd.read_stata(file)
    # Creates date variable based on file name
    df_temp['cohort'] = file[find_nth(file, '_', 2)+1:-4]
    # Create population count
    df_temp['population'] = 1
    li.append(df_temp)
    
df_input = pd.concat(li, axis=0, ignore_index=False).reset_index(drop=True)

In [4]:
# Do not count those who had sick notes beyond end dates
df_input.loc[df_input.sick_note == 0, 'first_sick_note_duration'] = np.nan

In [5]:
# Columns to subset
subset_cols = ['cohort','age_group','sex',
               'ethnicity','imd','region_string',
               'first_sick_note_duration',
               'sick_note','population']

# Subset to relevant columns and do not count those beyond 
df_clean = df_input[subset_cols].rename(columns={'region_string':'region'})

_____

## Median & IQR

In [6]:
def compute_med_iqr(path, demo=''):
    if demo == '':
        df_pct_ct = df_clean.groupby(
            ['cohort'])[['sick_note','population']].sum().reset_index()
        df_med = df_clean.groupby(
            ['cohort'])[['first_sick_note_duration']].apply(np.nanmedian).reset_index()
        df_pct25 = df_clean.groupby(
            ['cohort'])[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,25)).reset_index()
        df_pct75 = df_clean.groupby(
            ['cohort'])[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,75)).reset_index()
        dfs = [df_pct_ct, df_med, df_pct25, df_pct75]
        df_out = reduce(
            lambda left,right: pd.merge(left,right,on='cohort'), dfs
        ).rename(columns={'0_x':'median_duration','0_y':'pct25',0:'pct75'})
        df_out['iqr'] = df_out['pct75']-df_out['pct25']
    else:
        df_pct_ct = df_clean.groupby(
            ['cohort',demo])[['sick_note','population']].sum().reset_index()
        df_med = df_clean.groupby(
            ['cohort',demo])[['first_sick_note_duration']].apply(np.nanmedian).reset_index()
        df_pct25 = df_clean.groupby(
            ['cohort',demo])[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,25)).reset_index()
        df_pct75 = df_clean.groupby(
            ['cohort',demo])[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,75)).reset_index()
        dfs = [df_pct_ct, df_med, df_pct25, df_pct75]
        df_out = reduce(
            lambda left,right: pd.merge(left,right,on=['cohort',demo]), dfs
        ).rename(columns={'0_x':'median_duration','0_y':'pct25',0:'pct75'})
        df_out['iqr'] = df_out['pct75']-df_out['pct25']
    Path("../output/tabfig/").mkdir(parents=True, exist_ok=True)
    df_out.to_csv('../output/tabfig/' + path + '.csv', index=False)
    return df_out

In [7]:
# Overall
compute_med_iqr('med_iqr_overall')

,cohort,sick_note,population,median_duration,pct25,pct75,iqr
0,covid_2020,1166.0,1492,273.0,228.00,320.00,92.0
1,general_2019,7283.0,10000,269.0,222.00,318.00,96.0
2,general_2020,7879.0,10000,271.0,224.00,323.00,99.0
3,pneumonia_2019,378.0,504,268.5,223.25,321.75,98.5


In [8]:
# Age group
compute_med_iqr('med_iqr_age_group', 'age_group')

,cohort,age_group,sick_note,population,median_duration,pct25,pct75,iqr
0,covid_2020,0-17,108.0,138,272.0,224.00,315.50,91.50
1,covid_2020,18-24,130.0,150,275.5,211.50,328.25,116.75
2,covid_2020,25-34,102.0,134,286.5,241.75,318.25,76.50
3,covid_2020,35-44,127.0,160,266.0,232.50,317.50,85.00
4,covid_2020,45-54,235.0,307,265.0,219.50,317.50,98.00
5,covid_2020,55-69,249.0,324,278.5,233.00,331.00,98.00
6,covid_2020,70-79,112.0,145,265.5,228.00,317.00,89.00
7,covid_2020,80+,103.0,134,270.0,237.00,308.50,71.50
8,general_2019,0-17,772.0,1076,273.0,221.00,321.00,100.00
9,general_2019,18-24,697.0,937,266.0,224.00,310.00,86.00


In [9]:
# Sex
compute_med_iqr('med_iqr_sex', 'sex')

,cohort,sex,sick_note,population,median_duration,pct25,pct75,iqr
0,covid_2020,F,621.0,788,275.0,233.00,320.0,87.00
1,covid_2020,M,545.0,704,268.0,223.00,320.0,97.00
2,general_2019,F,3647.0,5006,270.0,222.00,317.0,95.00
3,general_2019,M,3636.0,4994,268.0,222.00,318.0,96.00
4,general_2020,F,4033.0,5129,270.0,224.00,322.0,98.00
5,general_2020,M,3846.0,4871,273.0,224.00,324.0,100.00
6,pneumonia_2019,F,182.0,252,265.0,215.50,320.5,105.00
7,pneumonia_2019,M,196.0,252,270.5,233.75,323.5,89.75


In [10]:
# Ethnicity
compute_med_iqr('med_iqr_ethnicity', 'ethnicity')

,cohort,ethnicity,sick_note,population,median_duration,pct25,pct75,iqr
0,covid_2020,White,678.0,877,272.0,227.00,323.00,96.00
1,covid_2020,Asian or Asian British,85.0,110,276.0,219.00,306.00,87.00
2,covid_2020,Other,103.0,125,275.0,220.00,321.00,101.00
3,covid_2020,Unknown,300.0,380,273.0,234.50,319.00,84.50
4,general_2019,White,4311.0,5925,267.0,222.00,316.00,94.00
5,general_2019,Asian or Asian British,570.0,792,272.0,225.00,319.00,94.00
6,general_2019,Other,560.0,783,267.0,220.00,315.25,95.25
7,general_2019,Unknown,1842.0,2500,273.0,224.00,320.00,96.00
8,general_2020,White,4716.0,5969,271.0,225.00,324.00,99.00
9,general_2020,Asian or Asian British,623.0,774,278.0,228.00,329.00,101.00


In [11]:
# IMD
compute_med_iqr('med_iqr_imd', 'imd')

,cohort,imd,sick_note,population,median_duration,pct25,pct75,iqr
0,covid_2020,0,64.0,76,277.5,235.75,333.25,97.50
1,covid_2020,1,229.0,298,270.0,232.75,312.25,79.50
2,covid_2020,2,237.0,306,272.0,235.00,323.00,88.00
3,covid_2020,3,216.0,275,273.0,220.50,323.00,102.50
4,covid_2020,4,203.0,264,275.0,227.50,324.00,96.50
5,covid_2020,5,217.0,273,273.0,218.00,317.00,99.00
6,general_2019,0,349.0,471,269.0,227.00,317.00,90.00
7,general_2019,1,1348.0,1846,269.0,219.00,317.50,98.50
8,general_2019,2,1363.0,1862,268.0,220.00,316.00,96.00
9,general_2019,3,1481.0,2005,269.0,225.00,312.00,87.00


In [12]:
# Region
compute_med_iqr('med_iqr_region', 'region')

,cohort,region,sick_note,population,median_duration,pct25,pct75,iqr
0,covid_2020,East,121.0,157,264.0,233.00,302.00,69.00
1,covid_2020,East Midlands,109.0,157,283.0,227.00,325.00,98.00
2,covid_2020,London,218.0,281,273.0,220.50,313.25,92.75
3,covid_2020,North East,128.0,149,283.0,224.75,337.00,112.25
4,covid_2020,North West,115.0,145,263.0,226.50,307.50,81.00
5,covid_2020,South East,110.0,144,273.0,227.25,339.00,111.75
6,covid_2020,South West,125.0,158,273.0,231.00,306.00,75.00
7,covid_2020,West Midlands,125.0,154,281.0,235.00,332.00,97.00
8,covid_2020,Yorkshire and The Humber,115.0,147,271.0,234.50,329.50,95.00
9,general_2019,East,735.0,1011,272.0,220.50,320.00,99.50
